In [6]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import os

In [7]:
# Load processed data
btc_df = pd.read_csv('processed_btc_data.csv')
eth_df = pd.read_csv('processed_eth_data.csv')
xrp_df = pd.read_csv('processed_xrp_data.csv')

# Convert Date column to datetime
btc_df['Date'] = pd.to_datetime(btc_df['Date'])
eth_df['Date'] = pd.to_datetime(eth_df['Date'])
xrp_df['Date'] = pd.to_datetime(xrp_df['Date'])

In [8]:
def create_lstm_model():
    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=(60, 1)),
        Dropout(0.2),
        LSTM(50, return_sequences=False),
        Dropout(0.2),
        Dense(25),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [ ]:
def train_and_forecast(df, crypto_name):
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(df['Close'].values.reshape(-1, 1))

    # Create sequences for training
    X_train, y_train = [], []
    for i in range(60, len(scaled_data) - 1):
        X_train.append(scaled_data[i - 60:i, 0])
        y_train.append(scaled_data[i, 0])
    
    X_train, y_train = np.array(X_train), np.array(y_train)
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

    # Train LSTM model
    model = create_lstm_model()
    model.fit(X_train, y_train, epochs=20, batch_size=32)

    # Predict next 5 years
    future_dates = [df['Date'].iloc[-1] + timedelta(days=i) for i in range(1, 5 * 365 + 1)]
    last_60_days = scaled_data[-60:].reshape(1, 60, 1)
    predictions = []

    for _ in range(5 * 365):
        pred = model.predict(last_60_days)[0][0]
        predictions.append(pred)
        last_60_days = np.append(last_60_days[:, 1:, :], [[[pred]]], axis=1)
    
    predictions = scaler.inverse_transform(np.array(predictions).reshape(-1, 1)).flatten()
    return pd.DataFrame({'Date': future_dates, f'{crypto_name}_Predicted': predictions})

# Generate Predictions
btc_preds = train_and_forecast(btc_df, 'BTC')
eth_preds = train_and_forecast(eth_df, 'ETH')
xrp_preds = train_and_forecast(xrp_df, 'XRP')

# Merge predictions
final_preds = btc_preds.merge(eth_preds, on='Date').merge(xrp_preds, on='Date')
final_preds.to_csv('crypto_predictions.csv', index=False)
print("Predictions saved successfully!")


C:\Users\ravis\OneDrive\Desktop\SAHIL_PROFILE\IMPORTANT\Cryptovision\venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 8s 47ms/step - loss: 0.0311
Epoch 2/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 3s 48ms/step - loss: 0.0024
Epoch 3/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 3s 48ms/step - loss: 0.0021
Epoch 4/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 3s 48ms/step - loss: 0.0017
Epoch 5/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 3s 48ms/step - loss: 0.0017
Epoch 6/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 3s 48ms/step - loss: 0.0017
Epoch 7/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 3s 48ms/step - loss: 0.0016
Epoch 8/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 3s 49ms/step - loss: 0.0014
Epoch 9/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 4s 52ms/step - loss: 0.0013
Epoch 10/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 3s 49ms/step - loss: 0.0011
Epoch 11/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 3s 48ms/step - loss: 0.0013
Epoch 12/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 3s 50ms/step - loss: 0.0012
Epoch 13/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 3s 47ms/step - loss: 0.0011    
Epoch 14/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 3s 46ms/step - loss: 9.9335e-04
Epoch 15/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 3s 48ms/step - loss: 0.0

C:\Users\ravis\OneDrive\Desktop\SAHIL_PROFILE\IMPORTANT\Cryptovision\venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


67/67 ━━━━━━━━━━━━━━━━━━━━ 8s 48ms/step - loss: 0.0112
Epoch 2/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 3s 48ms/step - loss: 0.0023
Epoch 3/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 3s 48ms/step - loss: 0.0019
Epoch 4/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 3s 47ms/step - loss: 0.0016
Epoch 5/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 3s 47ms/step - loss: 0.0013
Epoch 6/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 3s 47ms/step - loss: 0.0011    
Epoch 7/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 3s 47ms/step - loss: 0.0012
Epoch 8/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 3s 47ms/step - loss: 9.5217e-04
Epoch 9/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 3s 47ms/step - loss: 9.7587e-04
Epoch 10/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 3s 47ms/step - loss: 9.7301e-04
Epoch 11/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 3s 48ms/step - loss: 0.0012    
Epoch 12/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 3s 48ms/step - loss: 0.0010    
Epoch 13/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 3s 48ms/step - loss: 0.0013
Epoch 14/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 3s 47ms/step - loss: 8.4410e-04
Epoch 15/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 3s 47ms/step - 